In [359]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [360]:
# parent directory
parent_dir = Path.cwd().parent

# path to the dataset
data_path = os.path.join(parent_dir ,'data','raw','weekly-train-1.csv')

# reading a CSV file into a DataFrame
df = pd.read_csv(data_path)

In [361]:
df.head()

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_COUPON,IS_DISPLAY,IS_FEATURE,IS_PROMO,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY
0,Yogurt_Greek,3450014449,2020-12-28,20461.337740,5.667959,False,True,False,True,False,False,False,False,False,False,False,False,False
1,Yogurt_Greek,3450014449,2021-01-04,20366.054602,5.666967,False,True,False,True,False,False,False,False,False,False,False,False,False
2,Yogurt_Greek,3450014449,2021-01-11,18072.334581,5.667983,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Yogurt_Greek,3450014449,2021-01-18,18119.333530,5.667809,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Yogurt_Greek,3450014449,2021-01-25,18867.597360,5.040478,False,False,False,True,False,False,False,False,False,False,False,False,False


In [362]:
df.shape

(1016, 18)

In [363]:
# checking the data type of the 'DATE' column
df['DATE'].dtype.name

'str'

In [364]:
# converting the 'DATE' column to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])
print(df['DATE'].dtype.name)

datetime64[us]


In [365]:
# check if any rows have missing values
null_rows = df[df.isnull().any(axis=1)]
null_rows

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_COUPON,IS_DISPLAY,IS_FEATURE,IS_PROMO,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY


In [366]:
# converting boolean features into integers
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)
df.head()

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_COUPON,IS_DISPLAY,IS_FEATURE,IS_PROMO,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY
0,Yogurt_Greek,3450014449,2020-12-28,20461.337740,5.667959,0,1,0,1,0,0,0,0,0,0,0,0,0
1,Yogurt_Greek,3450014449,2021-01-04,20366.054602,5.666967,0,1,0,1,0,0,0,0,0,0,0,0,0
2,Yogurt_Greek,3450014449,2021-01-11,18072.334581,5.667983,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Yogurt_Greek,3450014449,2021-01-18,18119.333530,5.667809,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Yogurt_Greek,3450014449,2021-01-25,18867.597360,5.040478,0,0,0,1,0,0,0,0,0,0,0,0,0


## Feature Engineering
Re-arranging some features in order to be able to extract useful information

In [367]:
# summarizing holiday columns in one column
#holiday_cols  = ['IS_INDEPENDENCE_DAY', 'IS_LABOR_DAY', 'IS_MEMORIAL_DAY',
#                'IS_THANKSGIVING', 'IS_EASTER_SUNDAY',
#                 'IS_CHRISTMAS_DAY', 'IS_PRE_CHRISTMAS', 'IS_SUPER_BOWL_SUNDAY', 'IS_PRE_THANKSGIVING']

#df["IS_HOLIDAY_WEEK"] = df[holiday_cols].sum(axis=1)
#df.drop(columns=holiday_cols, inplace=True)
#df.head()

In [368]:
# creating a feature that indicates promotional level 
# NOTICE: this can be improved, eventually considering a weighted sum based on promo supports
df['PROMO_LEVEL'] = df['IS_PROMO'] + df['IS_COUPON'] + df['IS_DISPLAY'] + df['IS_FEATURE']
df.drop(columns=['IS_PROMO', 'IS_COUPON', 'IS_DISPLAY', 'IS_FEATURE'], inplace=True)
df.head()

,PRODUCT_DESC,PRODUCT_ID,DATE,TOTAL_SALES,PRICE,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY,PROMO_LEVEL
0,Yogurt_Greek,3450014449,2020-12-28,20461.337740,5.667959,0,0,0,0,0,0,0,0,0,2
1,Yogurt_Greek,3450014449,2021-01-04,20366.054602,5.666967,0,0,0,0,0,0,0,0,0,2
2,Yogurt_Greek,3450014449,2021-01-11,18072.334581,5.667983,0,0,0,0,0,0,0,0,0,0
3,Yogurt_Greek,3450014449,2021-01-18,18119.333530,5.667809,0,0,0,0,0,0,0,0,0,0
4,Yogurt_Greek,3450014449,2021-01-25,18867.597360,5.040478,0,0,0,0,0,0,0,0,0,1


In [369]:
# extracting time information from the 'DATE' column
# TO CHECK: is raw MONTH a good feature? sin and cos month?
df['MONTH'] = df['DATE'].dt.month

# TO CHECK: is WEEK_IN_MONTH a good feature? it seems a bit weak 
df['WEEK'] = df['DATE'].dt.isocalendar().week
df['WEEK_IN_MONTH'] = df['DATE'].dt.day.apply(lambda x: (x - 1) // 7 + 1)

df.drop(columns=['DATE','WEEK'], inplace=True)
df.head()

,PRODUCT_DESC,PRODUCT_ID,TOTAL_SALES,PRICE,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY,PROMO_LEVEL,MONTH,WEEK_IN_MONTH
0,Yogurt_Greek,3450014449,20461.337740,5.667959,0,0,0,0,0,0,0,0,0,2,12,4
1,Yogurt_Greek,3450014449,20366.054602,5.666967,0,0,0,0,0,0,0,0,0,2,1,1
2,Yogurt_Greek,3450014449,18072.334581,5.667983,0,0,0,0,0,0,0,0,0,0,1,2
3,Yogurt_Greek,3450014449,18119.333530,5.667809,0,0,0,0,0,0,0,0,0,0,1,3
4,Yogurt_Greek,3450014449,18867.597360,5.040478,0,0,0,0,0,0,0,0,0,1,1,4


In [370]:
df['PRODUCT_DESC'].unique()

<StringArray>
[            'Yogurt_Greek',        'Yogurt_Mini_Light',
           'Yogurt_Classic',      'Yogurt_Mini_Classic',
        'Yogurt_Mini_Greek',     'Yogurt_Giant_Classic',
 'Gold_Yogurt_Mini_Classic',   'Gold_Yogurt_Mini_Greek']
Length: 8, dtype: str

In [371]:
# turning product_id into a categorical variable
df['PRODUCT_ID'] = df['PRODUCT_ID'].astype('category').cat.codes # 'code' points to the real value in the categories' array
df.head()

,PRODUCT_DESC,PRODUCT_ID,TOTAL_SALES,PRICE,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY,PROMO_LEVEL,MONTH,WEEK_IN_MONTH
0,Yogurt_Greek,0,20461.337740,5.667959,0,0,0,0,0,0,0,0,0,2,12,4
1,Yogurt_Greek,0,20366.054602,5.666967,0,0,0,0,0,0,0,0,0,2,1,1
2,Yogurt_Greek,0,18072.334581,5.667983,0,0,0,0,0,0,0,0,0,0,1,2
3,Yogurt_Greek,0,18119.333530,5.667809,0,0,0,0,0,0,0,0,0,0,1,3
4,Yogurt_Greek,0,18867.597360,5.040478,0,0,0,0,0,0,0,0,0,1,1,4


## Preparing dataset for training
Splitting the dataset into training and test dataset. Standardizing numerical feature in order to enable better performances of our regression model.

In [372]:
X = df.drop(columns=['TOTAL_SALES', 'PRODUCT_DESC']) # dropping non-numeric and target columns
y = df['TOTAL_SALES']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [373]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((812, 14), (204, 14), (812,), (204,))

In [374]:
X_train

,PRODUCT_ID,PRICE,IS_INDEPENDENCE_DAY,IS_PRE_CHRISTMAS,IS_SUPER_BOWL_SUNDAY,IS_PRE_THANKSGIVING,IS_EASTER_SUNDAY,IS_LABOR_DAY,IS_MEMORIAL_DAY,IS_THANKSGIVING,IS_CHRISTMAS_DAY,PROMO_LEVEL,MONTH,WEEK_IN_MONTH
332,2,5.136625,1,0,0,0,0,0,0,0,0,0,6,4
708,5,4.874054,0,0,0,0,0,0,1,0,0,0,5,4
218,1,5.581567,0,0,0,0,0,0,0,0,0,2,9,4
425,3,4.990906,0,0,0,1,0,0,0,0,0,0,11,1
448,3,5.009982,0,0,0,0,1,0,0,0,0,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0,6.037204,0,0,0,0,0,0,0,0,0,0,1,2
270,2,4.803439,0,0,0,0,0,0,0,0,0,0,4,3
860,6,7.463875,0,0,0,0,0,0,0,1,0,1,11,2
435,3,5.011210,0,0,0,0,0,0,0,0,0,0,1,2


In [375]:
num_cols = ['PRICE']

#scaler = StandardScaler()
#X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
#X_test[num_cols] = scaler.transform(X_test[num_cols])

## Creating the Regression Model
We're gonna use XGBoost for this regression task.

In [376]:
# creating model
base_est = xgb.XGBRegressor()


param_grid = {
    "n_estimators": [300, 500],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2,3,4],
    "min_child_weight": [1, 3, 5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 0.3, 0.05],
    "reg_lambda": [1, 1.5],
    "reg_alpha": [0, 0.5],
}

'''
param_grid = {
    "n_estimators": [300, 500, 700],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [ 2,3,4],
    "min_child_weight": [1, 3, 5],
    "subsample": [ 0.8, 1.0],
    "colsample_bytree": [ 0.8, 1.0],
    "gamma": [0, 0.3, 0.5],
    "reg_alpha": [0, 0.5, 1],
    "reg_lambda": [1, 1.5, 2]
}
'''

gscv = GridSearchCV(estimator=base_est,
                    param_grid=param_grid,
                    scoring='neg_mean_squared_error',
                    cv=5, # 5-fold cross-validation
                    n_jobs=-1, # using all available cores
                    verbose=2)

gscv.fit(X_train, y_train)

xbm, xbm_params = gscv.best_estimator_, gscv.best_params_

print("Best parameters found: ", xbm_params)
print("Best score: ", -gscv.best_score_)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 4, 'min_child_weight': 5, 'n_estimators': 500, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.8}
Best score:  8333891.42539206


In [377]:
# average sales in the training set
avg_sales = y_train.mean()
print("Average sales in the training set: ", avg_sales)

# rmse
rmse = np.sqrt(-gscv.best_score_)
print("RMSE of the best model: ", rmse)

Average sales in the training set:  10736.71079730202
RMSE of the best model:  2886.848008709856


## Note:
- Should I convert BOOLEAN COLUMNS to int?
- Check what GPT is saying about the project. Why is RMSE so high rn?
- Remember to use MAPE as scoring!!